In [1]:
import os
import pandas as pd
import praw
import re
import hvplot.pandas
import time
from prawcore import TooManyRequests
import pandas as pd
from tqdm import tqdm
import langid
from openai import OpenAI
import tiktoken

from omegaconf import OmegaConf
from dotenv import load_dotenv
import openai
import pandas as pd
import asyncio
from tqdm.asyncio import tqdm_asyncio

In [2]:
parameters = OmegaConf.load("../../../parameters/reddit.yaml")
load_dotenv("../../../.env")

True

In [3]:
df = pd.read_csv("../../../datasets/reddit_multigec/pre_moderation_reddit_multigec.csv")
df

,title,text,language
0,Gilt investors warn Rachel Reeves she may need...,The yield rates are set by the gov agency and ...,english
1,‘I’ve earned £575 in six months’: The ex-Tory ...,The quality of politicians has declined hugely...,english
2,Firm job offers are being rescinded as well. I...,Ugh. Best of luck. I've turned my world upsi...,english
3,Train drivers to demand bigger pay deal in new...,Labour kind of opened the door for this when t...,english
4,Rachel Reeves to water down non-dom tax raid,"1. Have a ""think tank"" make up data\n2. Publis...",english
...,...,...,...
197729,Ринок найму в Україні,Я вам відповів чому працювати на казино для ме...,ukrainian
197730,Ринок найму в Україні,для цього мають бути окремі фонди міністерства...,ukrainian
197731,Ринок найму в Україні,"просто цікаво, а як ви відноситесь до людей як...",ukrainian
197732,Ринок найму в Україні,"Ні чуваче, це так не працює. Не можна виправдо...",ukrainian


In [5]:
tokenizer = tiktoken.encoding_for_model(parameters.tokenizer.model_name)
client = OpenAI(api_key=os.environ.get("OPEN_AI_API_KEY"))

In [9]:
import asyncio

def load_processed_data(output_csv):
    if os.path.exists(output_csv):
        return pd.read_csv(output_csv)
    else:
        return pd.DataFrame()

def get_unprocessed_data(df, processed_df, text_column):
    if not processed_df.empty:
        return df[~df[text_column].isin(processed_df[text_column])]
    else:
        return df

def append_to_csv(df, output_csv):
    if not os.path.exists(output_csv):
        df.to_csv(output_csv, mode='w', header=True, index=False)
    else:
        df.to_csv(output_csv, mode='a', header=False, index=False)

async def check_moderation_flag(expression):
    def sync_moderation_request(expression):
        moderation_response = client.moderations.create(
            model="omni-moderation-latest",
            input=expression,
        )
        return moderation_response.results[0].flagged

    flagged = await asyncio.to_thread(sync_moderation_request, expression)
    return flagged

async def moderate_content_in_batches(df, text_column, output_csv, batch_size=5000, delay=60):
    # Load already processed data
    processed_df = load_processed_data(output_csv)

    # Determine unprocessed data
    unprocessed_df = get_unprocessed_data(df, processed_df, text_column)

    if unprocessed_df.empty:
        print("All data has been processed.")
        return

    # Process data in batches
    for start in tqdm_asyncio(range(0, len(unprocessed_df), batch_size), desc="Processing batches"):
        end = start + batch_size
        batch_df = unprocessed_df.iloc[start:end]
        batch_texts = batch_df[text_column].tolist()

        tasks = [check_moderation_flag(text) for text in batch_texts]
        results = await tqdm_asyncio.gather(*tasks)

        batch_df['flagged'] = results

        # Append the batch results to the output CSV
        append_to_csv(batch_df, output_csv)

        await asyncio.sleep(delay)

output_csv = "../../../datasets/reddit_multigec/reddit_multigec.csv"
moderated_df = await moderate_content_in_batches(df, text_column='text', output_csv=output_csv)


  2%|▏         | 109/5000 [00:08<01:51, 43.70it/s]

100%|██████████| 5000/5000 [02:14<00:00, 37.13it/s]
/var/folders/gg/1kd93k4x47q5_gj92ljtjf000000gq/T/ipykernel_40275/1097438698.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['flagged'] = results
100%|██████████| 5000/5000 [02:14<00:00, 37.08it/s]
/var/folders/gg/1kd93k4x47q5_gj92ljtjf000000gq/T/ipykernel_40275/1097438698.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['flagged'] = results
100%|██████████| 5000/5000 [02:04<00:0

In [4]:
moderated_df = pd.read_csv("../../../datasets/reddit_multigec/reddit_multigec.csv")
moderated_df

,title,text,language,flagged
0,Gilt investors warn Rachel Reeves she may need...,The yield rates are set by the gov agency and ...,english,False
1,‘I’ve earned £575 in six months’: The ex-Tory ...,The quality of politicians has declined hugely...,english,False
2,Firm job offers are being rescinded as well. I...,Ugh. Best of luck. I've turned my world upsi...,english,False
3,Train drivers to demand bigger pay deal in new...,Labour kind of opened the door for this when t...,english,False
4,Rachel Reeves to water down non-dom tax raid,"1. Have a ""think tank"" make up data\n2. Publis...",english,False
...,...,...,...,...
197729,Ринок найму в Україні,Я вам відповів чому працювати на казино для ме...,ukrainian,False
197730,Ринок найму в Україні,для цього мають бути окремі фонди міністерства...,ukrainian,True
197731,Ринок найму в Україні,"просто цікаво, а як ви відноситесь до людей як...",ukrainian,False
197732,Ринок найму в Україні,"Ні чуваче, це так не працює. Не можна виправдо...",ukrainian,False


In [5]:
number_of_text_by_language_and_moderation = moderated_df.groupby(["language", "flagged"])["text"].count().rename("count").reset_index()
display(number_of_text_by_language_and_moderation)

number_of_text_by_language_and_moderation.loc[:, "flagged"] = number_of_text_by_language_and_moderation.loc[:, "flagged"].astype(int)
number_of_text_by_language_and_moderation = number_of_text_by_language_and_moderation.sort_values(by=["count", "flagged"])
number_of_text_by_language_and_moderation.hvplot.bar(
    x="language",
    by="flagged",
    grid=True,
    width=800,
    height=400,
)

,language,flagged,count
0,czech,False,16401
1,czech,True,3624
2,english,False,52898
3,english,True,7102
4,estonian,False,10551
5,estonian,True,872
6,german,False,44320
7,german,True,7625
8,greek,False,7851
9,greek,True,1920


:Bars   [language,flagged]   (count)

In [6]:
flagged_stats = moderated_df.groupby('language')['flagged'].agg(['sum', 'count']).reset_index()
flagged_stats['flagged_percentage'] = flagged_stats['sum'] / flagged_stats['count']
flagged_stats.sort_values(by="flagged_percentage", inplace=True, ascending=False)
flagged_stats

,language,sum,count,flagged_percentage
6,italian,3892,17213,0.226108
4,greek,1920,9771,0.196500
0,czech,3624,20025,0.180974
3,german,7625,51945,0.146790
8,slovene,1582,11655,0.135736
1,english,7102,60000,0.118367
10,ukrainian,516,5532,0.093275
7,latvian,624,7415,0.084154
2,estonian,872,11423,0.076337
9,swedish,111,2501,0.044382


In [7]:
flagged_stats.hvplot.bar(
    x="language",
    y="flagged_percentage",
    grid=True,
    width=800,
    height=400,
)

:Bars   [language]   (flagged_percentage)

In [8]:
multi_reddit_bronze_400_slim_df = moderated_df.loc[moderated_df.loc[:, "flagged"] == False]
multi_reddit_bronze_400_slim_df.to_csv("../../../datasets/reddit_multigec/reddit_multigec.csv", index=False)

multi_reddit_bronze_400_slim_df

,title,text,language,flagged
0,Gilt investors warn Rachel Reeves she may need...,The yield rates are set by the gov agency and ...,english,False
1,‘I’ve earned £575 in six months’: The ex-Tory ...,The quality of politicians has declined hugely...,english,False
2,Firm job offers are being rescinded as well. I...,Ugh. Best of luck. I've turned my world upsi...,english,False
3,Train drivers to demand bigger pay deal in new...,Labour kind of opened the door for this when t...,english,False
4,Rachel Reeves to water down non-dom tax raid,"1. Have a ""think tank"" make up data\n2. Publis...",english,False
...,...,...,...,...
197728,Ринок найму в Україні,"""Рятувати ідіотів самих від себе"" - ви в курсі...",ukrainian,False
197729,Ринок найму в Україні,Я вам відповів чому працювати на казино для ме...,ukrainian,False
197731,Ринок найму в Україні,"просто цікаво, а як ви відноситесь до людей як...",ukrainian,False
197732,Ринок найму в Україні,"Ні чуваче, це так не працює. Не можна виправдо...",ukrainian,False


In [9]:
multi_reddit_bronze_400_slim_df.loc[multi_reddit_bronze_400_slim_df.loc[:, "language"] == "ukrainian"]

,title,text,language,flagged
192199,Війна,А є якісь коротші терміни для: \n-російського...,ukrainian,False
192200,Війна,Ваше повідомлення вилучено. Заборонено поширюв...,ukrainian,False
192201,Війна,"Молодець що постиш інфу, ще й українською!",ukrainian,False
192203,Війна,Зробив добірку пісень котрі бодьорили нас та п...,ukrainian,False
192204,Війна,"Дякую, гарна збірка пісень! Сподіваємось на пе...",ukrainian,False
...,...,...,...,...
197728,Ринок найму в Україні,"""Рятувати ідіотів самих від себе"" - ви в курсі...",ukrainian,False
197729,Ринок найму в Україні,Я вам відповів чому працювати на казино для ме...,ukrainian,False
197731,Ринок найму в Україні,"просто цікаво, а як ви відноситесь до людей як...",ukrainian,False
197732,Ринок найму в Україні,"Ні чуваче, це так не працює. Не можна виправдо...",ukrainian,False


In [12]:
for sample in moderated_df.loc[(moderated_df.loc[:, "language"] == "ukrainian") & (moderated_df.loc[:, "flagged"] == True), "text"]:
    print(sample)

    print("---")

**Нагадування від команди модератоів:**

1. *Щодо емпатії та підтримки своїх побратимів/посестринців-українців:*

Cвоїх побратимів та посестринців українців варто всебічно підтримувати, особливо у такі важкі часи Московитсько-української війни. Для дурних розтлумачу: за поливання брудом інших українців й образи спрямовані на розкол суспільства на кшталт зеле/порохо/юле-тощо бот - буде видано бан.

Свою ж лють у коментарях варто виливати на московитів та їх колаборантів які ширять промосковитські та анти-українські наративи. Себто московитів та їх колаборантів ганити - це праведна справа, а от своїх українців ображати - це ганебно, бо у часи війни нам українцям варто всім триматися разом заради якнайшвидшої перемоги над Московією.

2. *Щодо використання правильної термінології при описі Московитсько-української війни*

З 2014 не було абстрактного АТО/ООС, а була Московитсько-українська війна. Те що українська влада побоялася називати речі своїми іменами у 2014 році аніскілечки не означа

In [11]:
multi_reddit_bronze_not_moderated_400_df = pd.read_csv("../../../datasets/reddit_multigec/reddit_multigec.csv")
multi_reddit_bronze_not_moderated_400_df.loc[multi_reddit_bronze_not_moderated_400_df.loc[:, "text"] == "Vyzkouseli jste Elocom?"]

,language,subreddit,is_selftext,is_submission,is_comment,is_reply,author,title,text,alphabetical_text,len_text,len_alphabetical_text,alphabetical_ratio,langid_language,translated_langid_language
26,czech,czech,False,False,False,True,evilbulk,"Litujete toho, že máte děti?",Vyzkouseli jste Elocom?,Vyzkouseli jste Elocom,23,22,0.956522,en,english


In [71]:
import langid

langid.classify("Vyzkouseli jste Elocom?")

('en', -26.93850612640381)